In [1]:
!pip install LangChain -q
!pip install -U langchain-community -q
!pip install pypdf -q
!pip install yt_dlp -q
!pip install pydub -q
!pip install unstructured -q
!pip install docx2txt -q
!pip install openai -q
!pip install python-dotenv -q
!pip install xlrd -q
!pip install tiktoken -q
!pip install chromadb -q
!pip install lark -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.9.0 requires keras-core, which is not installed.
keras-nlp 0.12.1 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 which is incompatible.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.2 which is incompatible.
jupyterlab 4.2.1 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
kfp 2.

In [2]:
from openai import OpenAI
import openai
import os
import numpy as np
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
openai.api_key = user_secrets.get_secret("openai_api")
client = OpenAI(
    # This is the default and can be omitted
    api_key=openai.api_key,
)

llm_model = "GPT-4o"

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("/kaggle/input/machine-learning-lectures/MachineLearning-Lecture01.pdf"),
    PyPDFLoader("/kaggle/input/machine-learning-lectures/MachineLearning-Lecture01.pdf"),
    PyPDFLoader("/kaggle/input/machine-learning-lectures/MachineLearning-Lecture02.pdf"),
    PyPDFLoader("/kaggle/input/machine-learning-lectures/MachineLearning-Lecture03.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

splits = text_splitter.split_documents(docs)
len(splits)
    
persist_directory = 'docs/chroma/'
!rm -rf ./docs/chroma  # remove old database files if any

vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

print(vectordb._collection.count())

/tmp/ipykernel_18/983296141.py:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)


209


In [4]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [5]:
from langchain.chat_models import ChatOpenAI

# Fetch OpenAI API key securely
user_secrets = UserSecretsClient()
OPENAI_API_KEY = user_secrets.get_secret("openai_api")

llm_model = "gpt-4" 

# Initialize the LLM
llm = ChatOpenAI(model_name=llm_model, temperature=0, openai_api_key=OPENAI_API_KEY)

/tmp/ipykernel_18/12768340.py:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name=llm_model, temperature=0, openai_api_key=OPENAI_API_KEY)


In [6]:
from langchain.chains import RetrievalQA

# Create the QA chain with the retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever()  # Ensure vectordb is properly set up
)

# Query the chain
result = qa_chain({"query": question})
print(result["result"])


/tmp/ipykernel_18/3777959211.py:10: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


The major topics for this class include statistics, algebra, and machine learning.


In [7]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [8]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)


In [9]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]


"Yes, basic probability and statistics are assumed knowledge for this class. However, if it's been a while since you've studied these topics, they will be reviewed in the discussion sections. Thanks for asking!"

In [10]:
result["source_documents"][0]

Document(metadata={'page': 4, 'page_label': '5', 'source': '/kaggle/input/machine-learning-lectures/MachineLearning-Lecture01.pdf'}, page_content="of this class will not be very program ming intensive, although we will do some \nprogramming, mostly in either MATLAB or Octa ve. I'll say a bit more about that later.  \nI also assume familiarity with basic proba bility and statistics. So most undergraduate \nstatistics class, like Stat 116 taught here at Stanford, will be more than enough. I'm gonna \nassume all of you know what ra ndom variables are, that all of you know what expectation \nis, what a variance or a random variable is. And in case of some of you, it's been a while \nsince you've seen some of this material. At some of the discussion sections, we'll actually \ngo over some of the prerequisites, sort of as  a refresher course under prerequisite class. \nI'll say a bit more about that later as well.  \nLastly, I also assume familiarity with basi c linear algebra. And again, mo

In [11]:
question = "Is probability a class topic?"

qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce",
)


In [12]:
result = qa_chain_mr({"query": question})
result["result"]


'The text does not provide information on probability being a class topic.'

In [13]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
result = qa_chain_mr({"query": question})
result["result"]


'Yes, probability is indeed a topic covered in this class. It is fundamental to understanding and interpreting linear regression and deriving learning algorithms. We will also be introducing a classification algorithm, which is particularly useful in scenarios with binary outcomes, such as medical diagnoses or predicting housing sales. For those who may need a refresher on related topics like statistics or algebra, we will cover these in the discussion sections. As the quarter progresses, we will also use these discussion sections to delve into extensions of the main lecture material, given the vastness of the field of machine learning.'

In [14]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)


In [15]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]

'Yes, basic probability and statistics are assumed knowledge for this class. The instructor assumes that all students know what random variables are, what expectation is, and what the variance of a random variable is. If some students need a refresher on these topics, they will be covered in the discussion sections.'

In [16]:
question = "why are those prerequesites needed?"
result = qa_chain({"query": question})
result["result"]


'The prerequisites are needed because the class involves understanding and applying complex machine learning algorithms. Basic knowledge of computer science is required to understand the programming and algorithmic aspects of the course. Familiarity with big O notation is important for understanding the time and space complexity of these algorithms. \n\nThe course also involves some programming, mostly in MATLAB or Octave, hence the need for basic computer skills. \n\nUnderstanding of basic probability and statistics is crucial because many machine learning algorithms are based on statistical concepts such as random variables, expectation, and variance. \n\nLastly, familiarity with basic linear algebra is needed because operations on matrices and vectors are fundamental in many machine learning algorithms. Knowing how to multiply matrices and vectors, understanding matrix inverses, and knowing what an eigenvector of a matrix is, are all important for understanding and implementing thes